In [ ]:
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'  # always print last expr.
%config InlineBackend.figure_format = 'svg'
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import tsdm
import tsdm.datasets.kiwi_runs_new

In [ ]:
ds = tsdm.datasets.kiwi_runs_new.KIWI_RUNS()

In [ ]:
ds.timeseries